In [1]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 796.3 kB/s eta 0:00:00a 0:00:01


In [2]:
import os, pyodbc
m_host = os.getenv('MSSQL_HOST', 'host.docker.internal')
m_port = os.getenv('MSSQL_PORT', '1433')
m_db   = os.getenv('MSSQL_DB', '')
m_user = os.getenv('MSSQL_USER', '')
m_pwd  = os.getenv('MSSQL_PASSWORD', '')
m_enc  = os.getenv('MSSQL_ENCRYPT', 'yes')
m_tsc  = os.getenv('MSSQL_TRUST_CERT', 'yes')
m_tds  = os.getenv('MSSQL_TDS_VERSION', '7.4')
conn_str = (    
    f"DRIVER={{FreeTDS}};SERVER={m_host};PORT={m_port};"    
    f"DATABASE={m_db};UID={m_user};PWD={m_pwd};"
    f"TDS_Version={m_tds};Encrypt={m_enc};TrustServerCertificate={m_tsc};"
)
print('ODBC connect string (safe):', conn_str.replace(m_pwd, '***'))
try:
    with pyodbc.connect(conn_str, timeout=5) as cx:
        cur = cx.cursor()
        cur.execute('SELECT @@VERSION')
        row = cur.fetchone()
        print('âœ… SQL Server connected. Version snippet:')
        print(str(row[0]).split('\n')[0])
        cur.execute('SELECT 1 AS one')
        print('Test query =>', cur.fetchone())
except Exception as e:
    print('âŒ SQL Server connection failed:', e)

ODBC connect string (safe): DRIVER={FreeTDS};SERVER=host.docker.internal;PORT=1433;DATABASE=DataCollection;UID=sa;PWD=***;TDS_Version=7.4;Encrypt=no;TrustServerCertificate=yes;
âœ… SQL Server connected. Version snippet:
Microsoft SQL Server 2022 (RTM) - 16.0.1000.6 (X64) 
Test query => (1,)


In [7]:
import pandas as pd
# Load the tables (if not already loaded)
df_completed_transects = pd.read_sql("SELECT * FROM dbo.CompletedTransects", cx)
df_completed_transects_info = pd.read_sql("SELECT * FROM dbo.CompletedTransectsInfo", cx)
df_completed_occurrences = pd.read_sql("SELECT * FROM dbo.CompletedOccurrences", cx)
df_completed_occurrences_info = pd.read_sql("SELECT * FROM dbo.CompletedOccurrencesInfo", cx)
df_completed_workflows = pd.read_sql("SELECT * FROM dbo.CompletedWorkflows", cx)
df_template_workflows = pd.read_sql("SELECT * FROM dbo.TemplateWorkflows", cx)
df_completed_responses = pd.read_sql("SELECT * FROM dbo.CompletedResponses", cx)

cx.close()
print("✅ SQL connection closed.")

/tmp/ipykernel_11134/1043501458.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_completed_transects = pd.read_sql("SELECT * FROM dbo.CompletedTransects", cx)
/tmp/ipykernel_11134/1043501458.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_completed_transects_info = pd.read_sql("SELECT * FROM dbo.CompletedTransectsInfo", cx)
/tmp/ipykernel_11134/1043501458.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_completed_occurrences = pd.read_sql("SELECT * FROM dbo.CompletedOccurrences", cx)
/tmp/ipykernel_11134/

✅ SQL connection closed.


In [10]:
# Save the DataFrames to a pickle file for later use
df_completed_transects.to_pickle("../data/pkl/df_completed_transects.pkl")
df_completed_transects_info.to_pickle("../data/pkl/df_completed_transects_info.pkl")
df_completed_occurrences.to_pickle("../data/pkl/df_completed_occurrences.pkl")
df_completed_occurrences_info.to_pickle("../data/pkl/df_completed_occurrences_info.pkl")
df_completed_workflows.to_pickle("../data/pkl/df_completed_workflows.pkl")
df_template_workflows.to_pickle("../data/pkl/df_template_workflows.pkl")
df_completed_responses.to_pickle("../data/pkl/df_completed_responses.pkl")

In [11]:
output_path = "../data/export/excel/raw_data.xlsx"

with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    df_completed_transects.to_excel(writer, sheet_name='completed_transects', index=False)
    df_completed_transects_info.to_excel(writer, sheet_name='completed_transects_info', index=False)
    df_completed_occurrences.to_excel(writer, sheet_name='completed_occurrences', index=False)
    df_completed_occurrences_info.to_excel(writer, sheet_name='completed_occurrences_info', index=False)
    df_completed_workflows.to_excel(writer, sheet_name='completed_workflows', index=False)
    df_template_workflows.to_excel(writer, sheet_name='template_workflows', index=False)
    df_completed_responses.to_excel(writer, sheet_name='completed_responses', index=False)

print(f"✅ Excel saved to: {output_path}")

✅ Excel saved to: ../data/export/excel/raw_data.xlsx


In [12]:
# Combine Pre_or_Post and QuestionText into a single column name
df_completed_transects_info['question_label'] = (
    df_completed_transects_info['Pre_or_Post'].str.strip() + ': ' +
    df_completed_transects_info['QuestionText'].str.strip()
)

# Pivot the info table to wide format
df_info_pivot = df_completed_transects_info.pivot_table(
    index='TransectUID',
    columns='question_label',
    values='Response',
    aggfunc='first'  # In case there are duplicates
).reset_index()

# Merge with the main CompletedTransects table
df_transects = df_completed_transects.merge(
    df_info_pivot,
    how='left',
    left_on='UID',
    right_on='TransectUID'
).drop(columns=['TransectUID'])  # Optional: remove duplicate join column

# Show result
df_transects.head()


,UID,Name,start_time,turn_time,end_time,lat_from,long_from,lat_turn,long_turn,lat_to,...,Post: Notes on Transect End,Pre: Ecomorphology habitat,Pre: New Transect?,Pre: Notes,Pre: Notes on Transect Start,Pre: OPC Vegetation,Pre: On old reserve?,Pre: Reserve vegetation,Pre: Transect physical habitat,Pre: Transect vegetation
0,271687,48,2011-07-08 16:09:19,2011-07-08 17:38:38,2011-07-08 17:53:34,0.019397,36.944255,0.019248,36.936132,0.019248,...,"ended at 900m, transect end near bushland",open/tree-less,Yes,start just south of road off grants plains,NaN,NaN,Yes,NaN,grass closed,grass
1,274498,49,2011-07-10 14:59:39,2011-07-10 15:21:22,2011-07-12 12:40:34,0.021680,36.941568,0.021160,36.935927,0.021160,...,NaN,open/tree-less,Yes,NaN,a bit north of road,NaN,Yes,NaN,grass closed,grass
2,277434,50,2011-07-12 15:59:36,2011-07-13 15:05:27,2011-07-13 15:56:27,0.022625,36.930758,0.022605,36.939808,0.022625,...,NaN,open/tree-less,Yes,NaN,heading east,NaN,Yes,NaN,grass closed,grass
3,278888,51,2011-07-13 16:09:31,2011-07-15 12:33:06,2011-07-15 12:47:30,0.023828,36.930668,0.023388,36.938127,0.023388,...,NaN,open/tree-less,Yes,NaN,,NaN,Yes,NaN,grass closed,grass
4,285685,52,2011-07-18 09:28:36,2011-07-18 10:31:19,2011-07-18 11:15:07,0.087638,36.828862,0.087870,36.837802,0.087870,...,3 patches of hyena poop,O;WBG,Yes,NaN,transitional between opn and wooded bushed gra...,NaN,No,NaN,grass closed,grass


In [14]:
# Create a combined label for each response column
df_completed_occurrences_info['question_label'] = (
    df_completed_occurrences_info['Pre_or_Post'].str.strip() + ': ' +
    df_completed_occurrences_info['QuestionText'].str.strip()
)

# Pivot to wide format
df_occurrence_info_pivot = df_completed_occurrences_info.pivot_table(
    index='OccurrenceID',
    columns='question_label',
    values='Response',
    aggfunc='first'
).reset_index()

# Merge with the main CompletedOccurrences table
df_occurrences = df_completed_occurrences.merge(
    df_occurrence_info_pivot,
    how='left',
    left_on='ID',
    right_on='OccurrenceID'
).drop(columns=['OccurrenceID'])  # Optional cleanup

# Show result
df_occurrences.head()

,ID,TransectUID,OccurrenceNumber,RecordingStartTime,RecordingEndTime,Lat,Long,Note,State,Post: Age,...,Post: Taxon guess?,Pre: Age,Pre: Distance spotted,Pre: Dried skin present?,Pre: Fresh bone(s)?,Pre: Local Context,Pre: Notes - fresh,Pre: Sex,Pre: Size class,Pre: Taxon
0,1,4354089,1,2003-02-17 09:12:35,2003-02-17 17:14:57,0.042495,36.970267,None,Completed,NaN,...,NaN,adult,0,No,No,"flat, open, short grass",,Unknown,3b-4,Bovidae (large)
1,2,4354089,2,2003-02-17 09:17:26,2003-02-17 17:20:34,0.042522,36.966980,None,Completed,NaN,...,NaN,adult,0,No,No,"very short grass, dirt patch, flat and open \n",,Unknown,4,eland
2,3,4354089,3,2003-02-17 09:55:37,2003-02-17 10:05:08,0.042432,36.966908,None,Completed,NaN,...,NaN,old adult,5,No,No,"flat, short grass",,Unknown,4,zebra
3,4,4354089,4,2003-02-17 10:14:26,2003-02-17 10:15:34,0.042495,36.966773,None,Completed,NaN,...,NaN,adult,0,No,No,"dirt patch, flat",,Unknown,0 - unknown,Mammalia indet
4,5,4354089,5,2003-02-17 10:30:03,2003-02-17 16:08:58,0.042549,36.966055,None,Completed,NaN,...,NaN,adult,16,No,No,"flat, very short grass",,Unknown,3,Bovidae (medium)


In [17]:
# Merge: Workflows + Occurrence context
df_instances = df_completed_workflows.merge(
    df_occurrences,
    how='left',
    left_on='OccurrenceID',
    right_on='ID',
    suffixes=('', '_occ')
)

# Merge: Add template workflow name
df_instances = df_instances.merge(
    df_template_workflows,
    how='left',
    left_on='TemplateWorkflowID',
    right_on='ID',
    suffixes=('', '_template')
).sort_values(
    by=['TransectUID', 'OccurrenceNumber', 'InstanceNumber']
).reset_index(drop=True)


# Optional cleanup (drop duplicate or ambiguous columns)
for col in ['ID_occ', 'ID_template']:
    if col in df_instances.columns:
        df_instances.drop(columns=col, inplace=True)

# Preview
df_instances.head()

,UID,OccurrenceID,TemplateWorkflowID,InstanceNumber,CompletedBy,ID,TransectUID,OccurrenceNumber,RecordingStartTime,RecordingEndTime,...,Pre: Dried skin present?,Pre: Fresh bone(s)?,Pre: Local Context,Pre: Notes - fresh,Pre: Sex,Pre: Size class,Pre: Taxon,Name,DateAdded,AddedBy
0,C3CD24C5-09B7-42DB-A5BE-82C862FCEE6A,265,AED98AE9-DD15-4449-B60D-C9F27A02F2EE,1,Briana,265.0,271687.0,1.0,2011-07-08 16:24:01,2011-07-08 16:30:47,...,Yes,Yes,in tall grass patch,dried skin bits,Unknown,3b,zebra,Bone,2019-10-15 00:42:13.370,nick
1,28FC916B-F859-444C-A3B6-24F1040AB291,265,AED98AE9-DD15-4449-B60D-C9F27A02F2EE,2,Briana,265.0,271687.0,1.0,2011-07-08 16:24:01,2011-07-08 16:30:47,...,Yes,Yes,in tall grass patch,dried skin bits,Unknown,3b,zebra,Bone,2019-10-15 00:42:13.370,nick
2,FF2ABC54-0172-4A03-81FE-72BAB7CE4C41,265,AED98AE9-DD15-4449-B60D-C9F27A02F2EE,3,Briana,265.0,271687.0,1.0,2011-07-08 16:24:01,2011-07-08 16:30:47,...,Yes,Yes,in tall grass patch,dried skin bits,Unknown,3b,zebra,Bone,2019-10-15 00:42:13.370,nick
3,43E88323-2CA0-4A29-8552-480BB96E4ACB,265,AED98AE9-DD15-4449-B60D-C9F27A02F2EE,4,Briana,265.0,271687.0,1.0,2011-07-08 16:24:01,2011-07-08 16:30:47,...,Yes,Yes,in tall grass patch,dried skin bits,Unknown,3b,zebra,Bone,2019-10-15 00:42:13.370,nick
4,24D9C6B4-FADC-4706-B09B-80EAD9F15D57,265,AED98AE9-DD15-4449-B60D-C9F27A02F2EE,5,Briana,265.0,271687.0,1.0,2011-07-08 16:24:01,2011-07-08 16:30:47,...,Yes,Yes,in tall grass patch,dried skin bits,Unknown,3b,zebra,Bone,2019-10-15 00:42:13.370,nick


In [18]:
df_briana = df_completed_workflows[df_completed_workflows['CompletedBy'].str.lower() == 'briana']
df_fire = df_completed_workflows[df_completed_workflows['CompletedBy'].str.lower() == 'fire']
# Clean up question text (optional)
df_completed_responses['response_label'] = df_completed_responses['QuestionText'].str.strip()

# Pivot: 1 row per instance, 1 column per question
df_response_pivot = df_completed_responses.pivot_table(
    index='CompletedWorkflowID',
    columns='response_label',
    values='Response',
    aggfunc='first'
).reset_index()

# Pivot the responses (common to both)
# Clean up question text (optional)
df_completed_responses['response_label'] = df_completed_responses['QuestionText'].str.strip()

# Pivot: 1 row per instance, 1 column per question
df_response_pivot = df_completed_responses.pivot_table(
    index='CompletedWorkflowID',
    columns='response_label',
    values='Response',
    aggfunc='first'
).reset_index()

# Merge Responses with Each User's Workflows
# Briana
df_briana_with_responses = df_briana.merge(
    df_response_pivot,
    how='left',
    left_on='UID',
    right_on='CompletedWorkflowID'
).drop(columns=['CompletedWorkflowID'])

# Fire
df_fire_with_responses = df_fire.merge(
    df_response_pivot,
    how='left',
    left_on='UID',
    right_on='CompletedWorkflowID'
).drop(columns=['CompletedWorkflowID'])

# Sort and Reset Index
df_briana_with_responses.sort_values(['OccurrenceID', 'InstanceNumber'], inplace=True)
df_briana_with_responses.reset_index(drop=True, inplace=True)

df_fire_with_responses.sort_values(['OccurrenceID', 'InstanceNumber'], inplace=True)
df_fire_with_responses.reset_index(drop=True, inplace=True)


In [19]:
# Preview
df_briana_with_responses.head()

,UID,OccurrenceID,TemplateWorkflowID,InstanceNumber,CompletedBy,Are there ecomorphable elements?,Articulated?,Bone notes,Buried - Notes,Buried?,...,UPM lo,UPM4 lc,UPM4 lo,UPM4 wc,UPM4 wo,Upper or lower?,Vertebra Portion,Weathering class,What element is this?,upper or lower?
0,B5E69EAE-E4FF-1B3E-F38B-0EC66DAC8774,1,AED98AE9-DD15-4449-B60D-C9F27A02F2EE,1,Briana,NaN,,,,No,...,NaN,NaN,NaN,NaN,NaN,NaN,,3,calcaneus,NaN
1,789F0CCC-7D1C-816F-F29C-B4E0BB3B82AB,2,AED98AE9-DD15-4449-B60D-C9F27A02F2EE,1,Briana,NaN,,damage but not sure if caused by carnivore,,No,...,NaN,NaN,NaN,NaN,NaN,NaN,,4,hemi-mandible,NaN
2,316C833E-D57B-C57D-6EB9-0F6447D46963,3,AED98AE9-DD15-4449-B60D-C9F27A02F2EE,1,Briana,NaN,,,,No,...,NaN,NaN,NaN,NaN,NaN,NaN,,1,hemi-mandible,NaN
3,17F84367-F24B-B574-F9CC-0CFF4F414BC0,3,AED98AE9-DD15-4449-B60D-C9F27A02F2EE,2,Briana,NaN,,,,No,...,NaN,NaN,NaN,NaN,NaN,NaN,,1,hemi-mandible,NaN
4,0FF21BFC-21B9-7915-45B2-5D82706B82B5,3,AED98AE9-DD15-4449-B60D-C9F27A02F2EE,3,Briana,NaN,,,,No,...,NaN,NaN,NaN,NaN,NaN,NaN,,1,hemi-mandible,NaN


In [20]:
# Preview
df_fire_with_responses.head()

,UID,OccurrenceID,TemplateWorkflowID,InstanceNumber,CompletedBy,Are there ecomorphable elements?,Articulated?,Bone notes,Buried - Notes,Buried?,...,UPM lo,UPM4 lc,UPM4 lo,UPM4 wc,UPM4 wo,Upper or lower?,Vertebra Portion,Weathering class,What element is this?,upper or lower?
0,A4802CC8-E5E0-439E-B6EC-38F3F46836D0,269,DE54448B-7AAD-4E55-B364-795A6F2888A7,3,Fire,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,B1AE1CD6-16E1-4C69-80B4-1DC02EBA1F0D,272,651BD177-976C-4D35-8BE9-8DFC3736BA9F,1,Fire,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,29428771-CDA7-48BA-87ED-6B52BDE512FB,273,651BD177-976C-4D35-8BE9-8DFC3736BA9F,1,Fire,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,36390049-5D37-44EB-AEFE-63651658463E,273,651BD177-976C-4D35-8BE9-8DFC3736BA9F,2,Fire,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EC8379E0-1868-4CD7-9130-89E546E91755,273,114A785B-804D-4AEB-892B-D6116DF31303,3,Fire,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df_transects.to_pickle("../data/pkl/df_transects.pkl")
df_occurrences.to_pickle("../data/pkl/df_occurrences.pkl")
df_instances.to_pickle("../data/pkl/df_instances.pkl")
df_briana_with_responses.to_pickle("../data/pkl/df_briana_with_responses.pkl")
df_fire_with_responses.to_pickle("../data/pkl/df_fire_with_responses.pkl")